In [2]:
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from PIL import Image
import os
import math

ModuleNotFoundError: No module named 'numpy'

In [ ]:
# def f(w:list, b:int, i:list[list]): # funcion normal. f 
#    return (math.tanh(w@i + b) + 1)/2

def f(w,b, i, d): # lagrangiano
   # funcion a minimizar
   sum = 0
   idx = 0
   for imagen in i:
      # (imagen=vector) 
      t_0:float = math.tanh(b+w@imagen) 
      sum += ((t_0 + 1) / 2 - d[idx])**2
      idx += 1
   return sum

def dfw(w,b, i:list[list], d:list):
   # derivada del lagrangiano contra w
   idx = 0
   sum = 0
   for imagen in i:
      # (imagen=vector) le queremos pasar una columna de i porque en la formula usa un vector (col de i)
      t_0:float = math.tanh(b+w@imagen) 
      sum += (1-t_0**2)*(((t_0+1)/2-d[idx]))*imagen
      idx += 1 
   return sum

def dfb(w,b, i:list[list], d:list):
   # derivada del lagrangiano contra b
   sum = 0
   idx = 0
   for imagen in i:
      # (imagen=vector) le queremos pasar una columna de i porque en la formula usa un vector (col de i)
      t_0:float = math.tanh(b+w@imagen) 
      sum += (1-t_0**2)*(((t_0+1)/2-d[idx]))
      idx += 1 
   return sum

def gradiente(x_t,i:list[list],d:list):
   # x es un punto de i_j (imagen aplanada = vector de tamaño escala*escala) (i: conjunto de imagenes)(i_j:imagen j del conjunto)
   # gradiente tiene una tupla: derivada de f contra b, derivada de f contra w.
   # n es la cant de columnas de la imagen == escala == sqrt(imagen.shape)
   # para acceder a la posicion x_t de una imagen aplanada hay que acceder a la pos (x_t[0]*(sqrt(imagen.shape)))+x_t[1]
      # ? d tmb es un dato... pero que?
   w = x_t[0]
   b = x_t[1]
   db = dfb(w,b,i,d)
   dw = dfw(w,b,i,d)
   return (db,dw) 
   
# x_0 es un punto aleatorio en Rn
# x_0 va a tener un valor aleatorio de b y un valor aleatorio de w
def descenso_por_gradiente(i, d):
   # función a optimizar
   K = len(i[0])
   # Multiplicar por 0.01 para reducir la escala de los valores (???)
   w_0 = np.random.randn(K) * 0.01  # w es un vector de R^K tq K=cant de píxeles en una imagen
   b_0 = np.random.randn() * 0.01
   x_t = (w_0, b_0)
   x_tsig = x_t
   alpha = 0.1
   TOLERANCIA = 0.0001
   MAX_ITER = 10000
   iter = 0
   while iter < MAX_ITER: # necesitamos esto porque no tenemos x_tsig al pcpio
      x_tsig = x_t - (alpha * gradiente(x_t, i, d))
      if abs(f(x_tsig, i, d) - f(x_t, i, d)) < TOLERANCIA:
         break
      x_t = x_tsig
      iter = iter + 1

   return x_tsig



In [ ]:
def abrirImagenesEscaladas(carpeta, escala=64):
    # abre todas las imagenes de la carpeta, y las escala de tal forma que midan (escala x escala)px
    # devuelve las imagenes aplanadas -> vectores de tamano escala^2 con valores entre 0 y 1
    imagenes = []

    for dirpath, dirnames, filenames in os.walk(carpeta):
        for file in filenames:
            img = Image.open( os.path.join(carpeta, file) )
            img = img.resize((escala, escala))
            img.convert('1')
            img = np.asarray(img)
            if len(img.shape)==3:
                img = img[:,:,0].reshape((escala**2 )) / 255
            else:
                img = img.reshape((escala**2 )) / 255
            
            imagenes.append( img )
    
    return imagenes

imagenes = abrirImagenesEscaladas('Instancia', 32)
print(imagenes[0]) # una imagen aplanada es un vector de escala*escala, no una matriz!!!
print(imagenes[0].shape)

[0.69019608 0.43921569 0.25490196 ... 0.11764706 0.12156863 0.1254902 ]
(1024,)


In [ ]:
import random
def balancear_datos(imagenes_entrenamiento):
    
    positivo = []
    negativo = []
    for imagen in imagenes_entrenamiento:
        if "bacteria" in imagen or "virus" in imagen:
            positivo.append(imagen)
        else:
            negativo.append(imagen)
            
    n = min(len(positivo), len(negativo))
    pos = positivo[:n]
    neg = negativo[:n]
    
    imagenes_entrenamiento_balanceadas = (pos+neg)
    random.shuffle(imagenes_entrenamiento_balanceadas)
    
    return imagenes_entrenamiento_balanceadas

normal = abrirImagenesEscaladas('./chest_ray/NORMAL', 64)
pneumonia = abrirImagenesEscaladas('./chest_ray/PNEUMONIA', 64)

lista_entera = normal + pneumonia

balancear_datos(lista_entera)

NameError: name 'abrirImagenesEscaladas' is not defined